# Zwicker Tone
The purpose of this notebook is to generate audio files capable of demonstrating the auditory illusion known as the Zwicker Tone as outlined in the article “Negative Afterimage” in Hearing from The Journal of the Acoustical Society of America (1964).

## Tools
To create the audio files several packages will be needed primarily `numpy` for generating white-noise signals and performing a Fourier transform, and `wav` for converting the numerical data into an audio file.